In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd # remove when finished
from implementations import *

In [2]:
data_tr = pd.read_csv('data/train.csv')
idx_tr = data_tr.Id

In [3]:
features_names = data_tr.columns.drop(['Id','Prediction'])
# features_names

In [4]:
def missing_rate(data):
    features = data.columns.drop(['Id','Prediction'])
    df_ = pd.DataFrame({'idx':[np.nan]})
    for feature in features:
        data2 = data.loc[(data[feature]==-999)].copy()
        missing_rate = data2[feature].size/data[feature].size
        df1 = pd.DataFrame({feature:[missing_rate]})
        df_ = pd.concat([df_,df1],axis=1)
    df_ = df_.drop(columns='idx')
    df_ = df_.T.rename(columns={0:'rate'})
    return df_

In [5]:
def drop_columns(df_miss):
    names_70 = list(df_miss.loc[df_miss.rate==0.709828].index)
    names_30 = list(df_miss.loc[df_miss.rate==0.399652].index)
    names_phi = list(['PRI_jet_leading_phi','PRI_jet_subleading_phi','PRI_lep_phi','PRI_met_phi','PRI_tau_phi','PRI_jet_num'])
    df_drop_name = {'names_70':names_70,'names_30':names_30,'names_phi':names_phi}
    return df_drop_name

In [6]:
df_miss_tr = missing_rate(data_tr)
df_miss_tr

,rate
DER_mass_MMC,0.152456
DER_mass_transverse_met_lep,0.000000
DER_mass_vis,0.000000
DER_pt_h,0.000000
DER_deltaeta_jet_jet,0.709828
DER_mass_jet_jet,0.709828
DER_prodeta_jet_jet,0.709828
DER_deltar_tau_lep,0.000000
DER_pt_tot,0.000000
DER_sum_pt,0.000000


In [7]:
# replacing the -999 with mode in signal or background

DER_mode_s = data_tr.loc[(data_tr['DER_mass_MMC']!=-999)].loc[(data_tr['Prediction'])=='s']['DER_mass_MMC'].mean(axis=0)#value_counts().index[0]
DER_mode_b = data_tr.loc[(data_tr['DER_mass_MMC']!=-999)].loc[(data_tr['Prediction'])=='b']['DER_mass_MMC'].mean(axis=0)#value_counts().index[0]

idx_s = data_tr.loc[(data_tr['DER_mass_MMC']==-999)].loc[(data_tr['Prediction'])=='s']['DER_mass_MMC'].index
idx_b = data_tr.loc[(data_tr['DER_mass_MMC']==-999)].loc[(data_tr['Prediction'])=='b']['DER_mass_MMC'].index

data_tr.loc[idx_s,'DER_mass_MMC']=DER_mode_s
data_tr.loc[idx_b,'DER_mass_MMC']=DER_mode_b


In [8]:
drop_col = drop_columns(df_miss_tr)
drop_col

{'names_70': ['DER_deltaeta_jet_jet',
  'DER_mass_jet_jet',
  'DER_prodeta_jet_jet',
  'DER_lep_eta_centrality',
  'PRI_jet_subleading_pt',
  'PRI_jet_subleading_eta',
  'PRI_jet_subleading_phi'],
 'names_30': ['PRI_jet_leading_pt',
  'PRI_jet_leading_eta',
  'PRI_jet_leading_phi'],
 'names_phi': ['PRI_jet_leading_phi',
  'PRI_jet_subleading_phi',
  'PRI_lep_phi',
  'PRI_met_phi',
  'PRI_tau_phi',
  'PRI_jet_num']}

'DER_deltaeta_jet_jet','DER_mass_jet_jet','DER_prodeta_jet_jet','DER_lep_eta_centrality','PRI_jet_subleading_pt','PRI_jet_subleading_eta','PRI_jet_subleading_phi': missing rate = 70.9% (PRI_jet_num <= 1)

'PRI_jet_leading_pt','PRI_jet_leading_eta','PRI_jet_leading_phi': missing rate = 39.9% (PRI_jet_num = 0)

In [9]:
def group_features_by_jet(df):
    return {  
        0: df.loc[df['PRI_jet_num'] == 0].copy(),
        1: df.loc[df['PRI_jet_num'] == 1].copy(),
        2: df.loc[(df['PRI_jet_num'] == 2) | (df['PRI_jet_num'] == 3)].copy(),
    }

In [10]:
feature_tr_grp = group_features_by_jet(data_tr)

In [11]:
feature_tr_grp[0] = feature_tr_grp[0].drop(columns = list(set(drop_col['names_phi']+drop_col['names_70']+drop_col['names_30'])))
feature_tr_grp[0] = feature_tr_grp[0].drop(columns = 'PRI_jet_all_pt')# all 0s
feature_tr_grp[1] = feature_tr_grp[1].drop(columns = list(set(drop_col['names_phi']+drop_col['names_70'])))
feature_tr_grp[2] = feature_tr_grp[2].drop(columns = list(set(drop_col['names_phi'])))

In [12]:
# for df['PRI_jet_num'] == 0
x_tr0,mean0,std0 = standardize(feature_tr_grp[0].drop(columns = ['Id','Prediction']).values)
y_tr0 = feature_tr_grp[0].Prediction.replace(['s','b'],[1,-1]).values
w0 = least_squares(y_tr0,x_tr0)
idx_tr0 = feature_tr_grp[0].Id
y_pred_tr0 = predict_labels(w0, x_tr0)


In [13]:
# for df['PRI_jet_num'] == 1
x_tr1,_,_ = standardize(feature_tr_grp[1].drop(columns = ['Id','Prediction']).values)
y_tr1 = feature_tr_grp[1].Prediction.replace(['s','b'],[1,-1]).values
w1 = least_squares(y_tr1,x_tr1)
idx_tr1 = feature_tr_grp[1].Id
y_pred_tr1 = predict_labels(w1, x_tr1)


In [14]:
# for df['PRI_jet_num'] == 2,3
x_tr2,_,_ = standardize(feature_tr_grp[2].drop(columns = ['Id','Prediction']).values)
y_tr2 = feature_tr_grp[2].Prediction.replace(['s','b'],[1,-1]).values
w2 = least_squares(y_tr2,x_tr2)
idx_tr2 = feature_tr_grp[2].Id
y_pred_tr2 = predict_labels(w2, x_tr2)

In [15]:
# create_csv_submission(idx_te, y_pred_te, 'submission.csv')

In [24]:
k_fold = 10
seed = 1
lambdas = np.logspace(-10, 2, 20) 

for lambda_ in lambdas:
    acc_tr0,acc_te0 = cv_loop(y_tr0, x_tr0,k_fold,seed,ridge_regression,lambda_=lambda_)
    print("lambda = {}: Training accuracy: {} - Test accuracy : {}".format(lambda_, acc_tr0, acc_te0))

lambda = 1e-10: Training accuracy: 0.8167923311314251 - Test accuracy : 0.816634971474327
lambda = 4.281332398719396e-10: Training accuracy: 0.8167834345321501 - Test accuracy : 0.816604944450005
lambda = 1.8329807108324374e-09: Training accuracy: 0.8167845466070596 - Test accuracy : 0.8166149534581123
lambda = 7.847599703514623e-09: Training accuracy: 0.8167834345321501 - Test accuracy : 0.8166249624662196
lambda = 3.3598182862837814e-08: Training accuracy: 0.8167845466070596 - Test accuracy : 0.8166249624662196
lambda = 1.438449888287663e-07: Training accuracy: 0.8167845466070596 - Test accuracy : 0.8166249624662196
lambda = 6.158482110660254e-07: Training accuracy: 0.8167856586819688 - Test accuracy : 0.8166249624662196
lambda = 2.6366508987303555e-06: Training accuracy: 0.8167889949066968 - Test accuracy : 0.8166249624662196
lambda = 1.1288378916846883e-05: Training accuracy: 0.8167912190565156 - Test accuracy : 0.8166149534581123
lambda = 4.8329302385717524e-05: Training accuracy:

In [25]:
for lambda_ in lambdas:
    acc_tr1,acc_te1 = cv_loop(y_tr1, x_tr1,k_fold,seed,ridge_regression,lambda_=lambda_)
    print("lambda = {}: Training accuracy: {} - Test accuracy : {}".format(lambda_, acc_tr1, acc_te1))

lambda = 1e-10: Training accuracy: 0.710692076228686 - Test accuracy : 0.7103946350270828
lambda = 4.281332398719396e-10: Training accuracy: 0.7107235993695372 - Test accuracy : 0.7104075315965953
lambda = 1.8329807108324374e-09: Training accuracy: 0.7107235993695371 - Test accuracy : 0.7103559453185453
lambda = 7.847599703514623e-09: Training accuracy: 0.7107278979796533 - Test accuracy : 0.7103430487490328
lambda = 3.3598182862837814e-08: Training accuracy: 0.7107264651096146 - Test accuracy : 0.7103430487490328
lambda = 1.438449888287663e-07: Training accuracy: 0.7107264651096146 - Test accuracy : 0.7103430487490328
lambda = 6.158482110660254e-07: Training accuracy: 0.7107264651096146 - Test accuracy : 0.7103430487490328
lambda = 2.6366508987303555e-06: Training accuracy: 0.7107264651096146 - Test accuracy : 0.7103430487490328
lambda = 1.1288378916846883e-05: Training accuracy: 0.7107278979796533 - Test accuracy : 0.7103430487490328
lambda = 4.8329302385717524e-05: Training accuracy

In [23]:
for lambda_ in lambdas:
    acc_tr2,acc_te2 = cv_loop(y_tr2, x_tr2,k_fold,seed,ridge_regression,lambda_=lambda_)
    print("lambda = {}: Training accuracy: {} - Test accuracy : {}".format(lambda_, acc_tr2, acc_te2))

lambda = 1e-20: Training accuracy: 0.7308811591539157 - Test accuracy : 0.7307003032809485
lambda = 1.4384498882876659e-19: Training accuracy: 0.7308811591539157 - Test accuracy : 0.7307003032809485
lambda = 2.0691380811147986e-18: Training accuracy: 0.7308811591539157 - Test accuracy : 0.7307003032809485
lambda = 2.976351441631313e-17: Training accuracy: 0.7308811591539157 - Test accuracy : 0.7307003032809485
lambda = 4.281332398719396e-16: Training accuracy: 0.7308811591539157 - Test accuracy : 0.7307003032809485
lambda = 6.15848211066028e-15: Training accuracy: 0.7308811591539157 - Test accuracy : 0.7307003032809485
lambda = 8.858667904100832e-14: Training accuracy: 0.7308842224570755 - Test accuracy : 0.7307003032809485
lambda = 1.2742749857031321e-12: Training accuracy: 0.7309117921855136 - Test accuracy : 0.7307278742762614
lambda = 1.8329807108324375e-11: Training accuracy: 0.7309715265971296 - Test accuracy : 0.7307554452715743
lambda = 2.6366508987303664e-10: Training accuracy